**step 1: install torchdata**

Remove PyTorch first -- torchdata depends on a nightly version of PyTorch.

In [9]:
!pip uninstall torch

Found existing installation: torch 1.11.0
Uninstalling torch-1.11.0:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/bin/torchrun
    /usr/local/lib/python3.7/dist-packages/caffe2/*
    /usr/local/lib/python3.7/dist-packages/torch-1.11.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/torch/*
Proceed (y/n)? y
  Successfully uninstalled torch-1.11.0


In [10]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


Install a PyTorch nightly binary. Pick a different one depending on the CUDA version:

In [11]:
# For CUDA 10.2
# >>> pip install --pre torch -f https://download.pytorch.org/whl/nightly/cu102/torch_nightly.html
# For CUDA 11.1
# >>> pip install --pre torch -f https://download.pytorch.org/whl/nightly/cu111/torch_nightly.html
# For CPU-only build
# >>> pip install --pre torch -f https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html

# We're going to use a CPU-only binary
!pip install --pre torch -f https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html

Looking in links: https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html
     |████████████████████████████████| 189.0 MB 24 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.13.0.dev20220521+cpu which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.13.0.dev20220521+cpu which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.13.0.dev20220521+cpu which is incompatible.


Install torchdata.

In [12]:
!pip install --user "git+https://github.com/pytorch/data.git"

  Cloning https://github.com/pytorch/data.git to /tmp/pip-req-build-4ge2jr3_
  Running command git clone -q https://github.com/pytorch/data.git /tmp/pip-req-build-4ge2jr3_
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 138 kB 32.7 MB/s 
     |████████████████████████████████| 127 kB 50.9 MB/s 
  Created wheel for torchdata: filename=torchdata-0.4.0a0+12cfaf8-py3-none-any.whl size=87747 sha256=d344fee6093e5f822fd20a7f9ec5daa2300db9e9f89d0cd8b9610116097bfb29
  Stored in directory: /tmp/pip-ephem-wheel-cache-8_2uhnqc/wheels/a9/4f/7a/6edd9e915eb0cf0d88565db0f14b9b2b83e5f9f56c192042ad
Successfully built torchdata
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires f

**When this is done, you may need to restart the colab runtime to complete the installation. Go to Runtime > Restart runtime**

Finally, run some sanity checks.

In [1]:
from torchdata.datapipes.iter import HttpReader
URL = "https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv"
ag_news_train = HttpReader([URL]).parse_csv().map(lambda t: (int(t[0]), " ".join(t[1:])))
agn_batches = ag_news_train.batch(2).map(lambda batch: {'labels': [sample[0] for sample in batch],\
                                      'text': [sample[1].split() for sample in batch]})
batch = next(iter(agn_batches))
print(batch)
assert batch['text'][0][0:8] == ['Wall', 'St.', 'Bears', 'Claw', 'Back', 'Into', 'the', 'Black']

{'labels': [3, 3], 'text': [['Wall', 'St.', 'Bears', 'Claw', 'Back', 'Into', 'the', 'Black', '(Reuters)', 'Reuters', '-', 'Short-sellers,', 'Wall', "Street's", 'dwindling\\band', 'of', 'ultra-cynics,', 'are', 'seeing', 'green', 'again.'], ['Carlyle', 'Looks', 'Toward', 'Commercial', 'Aerospace', '(Reuters)', 'Reuters', '-', 'Private', 'investment', 'firm', 'Carlyle', 'Group,\\which', 'has', 'a', 'reputation', 'for', 'making', 'well-timed', 'and', 'occasionally\\controversial', 'plays', 'in', 'the', 'defense', 'industry,', 'has', 'quietly', 'placed\\its', 'bets', 'on', 'another', 'part', 'of', 'the', 'market.']]}


In [2]:
!pip install -U torchtext==0.10.0

     |████████████████████████████████| 7.6 MB 25.5 MB/s 
     |████████████████████████████████| 831.4 MB 2.1 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0.dev20220521+cpu
    Uninstalling torch-1.13.0.dev20220521+cpu:
      Successfully uninstalled torch-1.13.0.dev20220521+cpu
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.12.0
    Uninstalling torchtext-0.12.0:
      Successfully uninstalled torchtext-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.4.0a0+12cfaf8 requires torch>1.11.0, but you have torch 1.9.0 which is incompatible.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.9.0 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.9.0 which is incompatible.


**restart here again**

**step 2: access to the raw dataset iterators**

In [1]:
%matplotlib inline

In [2]:
import torch
from torchtext.datasets import AG_NEWS
train_iter = iter(AG_NEWS(split='train'))

train.csv: 29.5MB [00:00, 85.5MB/s]


In [3]:
next(train_iter)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

**step 3: Prepare data processing pipelines**

In [4]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
train_iter = AG_NEWS(split='train')

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [5]:
vocab(['here', 'is', 'an', 'example'])

[475, 21, 30, 5297]

In [6]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

In [7]:
text_pipeline('here is the an example')

[475, 21, 2, 30, 5297]

In [8]:
label_pipeline('10')

9

**step 4: Generate data batch and iterator**

In [9]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter = AG_NEWS(split='train')
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

**step 5: Define the model**

In [10]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

**step 5: Initiate an instance**

In [11]:
train_iter = AG_NEWS(split='train')
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

**step 6: Define functions to train the model and evaluate results.**

In [12]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

**step 7: Split the dataset and run the model**

In [13]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = AG_NEWS()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

test.csv: 1.86MB [00:00, 74.5MB/s]                  


| epoch   1 |   500/ 1782 batches | accuracy    0.685
| epoch   1 |  1000/ 1782 batches | accuracy    0.855
| epoch   1 |  1500/ 1782 batches | accuracy    0.875
-----------------------------------------------------------
| end of epoch   1 | time: 16.29s | valid accuracy    0.882 
-----------------------------------------------------------
| epoch   2 |   500/ 1782 batches | accuracy    0.897
| epoch   2 |  1000/ 1782 batches | accuracy    0.900
| epoch   2 |  1500/ 1782 batches | accuracy    0.905
-----------------------------------------------------------
| end of epoch   2 | time: 15.26s | valid accuracy    0.885 
-----------------------------------------------------------
| epoch   3 |   500/ 1782 batches | accuracy    0.914
| epoch   3 |  1000/ 1782 batches | accuracy    0.914
| epoch   3 |  1500/ 1782 batches | accuracy    0.916
-----------------------------------------------------------
| end of epoch   3 | time: 15.57s | valid accuracy    0.901 
-------------------------------

**step 8: Evaluate the model with test dataset**

In [14]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.905


**step 9: Test on a random news**

In [23]:
ag_news_label = {1: "World",
                 2: "Sports",
                 3: "Business",
                 4: "Sci/Tec"}

def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

model = model.to("cpu")

print("This is a %s news" %ag_news_label[predict(ex_text_str, text_pipeline)])

This is a Sports news


In [24]:
ex_text_str2 = "A tornado left 43 people injured as it cut a path of destruction through several towns in western Germany, police said. \
Officers in the city of Paderborn said the tornado ripped off roofs and debris was strewn around for kilometres. \
Ten people have serious injuries and one woman's life is in danger, they said. \
A 38-year-old man also died in severe storms which lashed the region on Friday."

print("This is a %s news" %ag_news_label[predict(ex_text_str2, text_pipeline)])

This is a World news


In [25]:
ex_text_str3 = "Canada says it will ban two of China's biggest telecoms equipment makers from working on its 5G phone networks. \
The restrictions against Huawei and ZTE were announced by the country's industry minister on Thursday. \
Francois-Philippe Champagne says the move will improve Canada's mobile internet services and protect the safety and security of Canadians. \
But Huawei Canada said it was disappointed by the decision, which it said was political."

print("This is a %s news" %ag_news_label[predict(ex_text_str3, text_pipeline)])

This is a Sci/Tec news


In [26]:
ex_text_str4 = "The lure of making a quick buck has always attracted young people to invest in risky assets. \
For Generation Z, it is the volatility - and the decentralised nature - of digital assets such as cryptocurrency and NFTs which appeals. \
But they are unregulated, meaning there is little investor protection. \
All my friends were talking about [cryptocurrency] so one day I just decided why not just jump in and see if I can make some money, says 20-year-old Paxton See Tow. \
All he needed was his phone and trading thousands of dollars' worth of assets was only a click away."

print("This is a %s news" %ag_news_label[predict(ex_text_str4, text_pipeline)])

This is a Business news


In [27]:
ex_text_str5 = "Automakers from Tesla to Rivian to Cadillac are hiking prices on their electric vehicles \
amid changing market conditions and rising commodity costs, specifically for key materials needed for EV batteries.\
Battery prices have been declining for years, but that may be about to change. \
One firm projects a sharp increase in demand for battery minerals over the next \
four years that could push the price of EV battery cells up by more than 20%. \
That’s on top of already-rising prices for battery-related raw materials, a result \
of supply-chain disruptions related to Covid and Russia’s invasion of Ukraine."

print("This is a %s news" %ag_news_label[predict(ex_text_str5, text_pipeline)])

This is a Business news
